In [ ]:
import numpy as np
from sympy import Matrix, symbols, Poly, invert, gcd

In [ ]:


def poly_mult_mod(a, b, N, q):
    """Multiplies two polynomials a and b modulo q and X^N - 1"""

    result = np.zeros(N, dtype=int)
    for i in range(N):
        for j in range(N):
            result[(i + j) % N] += (a[i] * b[j])
    print("Print polynomial Multiplication: ",result%q)
    return result%q

def poly_inv_mod(f, N, q):
    """Finds the inverse of a polynomial f mod q using SymPy's invert function."""
    x = symbols('x')

    f_poly = Poly(f, x).set_modulus(q)
    #print(f_poly)
    modulus_poly = Poly(x**N - 1, x).set_modulus(q)

    try:
        # Check if the polynomial is invertible (gcd(f, x^N - 1) should be 1)
        if gcd(f_poly, modulus_poly) != 1:
            print(f"Polynomial {f} is not invertible mod {q}")
            return None

        f_inv_poly = invert(f_poly, modulus_poly)  # Finds the modular inverse
        f_inv = np.array(f_inv_poly.all_coeffs(), dtype=int) % q  # Convert to NumPy array


        # Ensure the resulting inverse has N coefficients
        if len(f_inv) < N:
            f_inv = np.concatenate([np.zeros(N - len(f_inv)), f_inv])  # Pad with zeros
        return f_inv
    except Exception as e:
        print(f"Error computing inverse for polynomial {f}: {e}")
        return None  # Return None if the inverse doesn't exist

def random_ternary_poly(d1, d2, N):
    """Generates a random ternary polynomial with exactly d1 coefficients equal to 1 and d2 equal to -1."""
    poly = [0] * N
    ones = np.random.choice(N, d1, replace=False)  # Select positions for +1s
    minus_ones = np.random.choice(list(set(range(N)) - set(ones)), d2, replace=False)  # Select positions for -1s

    for i in ones:
        poly[i] = 1
    for i in minus_ones:
        poly[i] = -1

    return poly

def keygen(N, p, q):
    """Key generation: creates the public key h and private keys f, Fp, Fq"""
    while True:
        # Randomly choose f from T(d+1, d) and g from T(d, d)
        #f=[-1,0,1,1,-1,0,1]
        f = random_ternary_poly(d1=N//3+1, d2=N//3, N=N)
        g = random_ternary_poly(d1=N//3, d2=N//3, N=N)
        print("f: ",f)
        print("g: ",g)
        buffer_f=f
        buffer_f.reverse()
        #g=[0,-1,-1,0,1,0,1]

        Fq = poly_inv_mod(buffer_f, N, q)
        Fq=np.flip(Fq)
        print("Inverse of f mod q (Fq):", Fq)
        Fp = poly_inv_mod(buffer_f, N, p)
        Fp=np.flip(Fp)
        print("Inverse of f mod p (Fp):", Fp)

        if Fq is not None and Fp is not None:
            break

    h = p*poly_mult_mod(Fq, g, N, q)%q
    return f, Fp, Fq, h

def encrypt(h, m, r, N, p, q):
    """Encrypts message m using public key h and random r"""


    pr = poly_mult_mod(h, r, N, q)  # p * r
    e = (pr + m) % q  # e = p*r + h*m mod q
    return e

def decrypt(e, f, Fp, N, p, q):
  """Decrypts encrypted message e using private keys f and Fp"""
  a = poly_mult_mod(f, e, N, q)  # Multiply f and e mod q
  a = np.array([(x if x <= q // 2 else x - q) for x in a])  # Coefficients in [-q/2, q/2]

  # Multiply Fp and a mod p
  m_recovered = poly_mult_mod(Fp, a, N, p) % p
  # Ensure m_recovered values are in [-p/2, p/2]
  m_recovered = np.array([(x if x <= p // 2 else x - p) for x in m_recovered])
  return m_recovered

# Example usage
N = 7   # Degree of the polynomials (choose small for example)
p = 3   # Small modulus
q = 41  # Large modulus

# Key generation
f, Fp, Fq, h = keygen(N, p, q)
print("Public key (h):", h)
f.reverse()
print("Private key (f):", f)

# Encryption
m = np.array([1, 0, 1, 1, 0, -1, 0])  # Example message as a polynomial
r = random_ternary_poly(d1=N//3, d2=N//3, N=N)  # Random polynomial r
print("r: ",r)
#r=[-1,1,0,0,0,-1,1]
e = encrypt(h, m, r, N, p, q)
print("Encrypted message (e):", e)


# Decryption
m_recovered = decrypt(e, f, Fp, N, p, q)
print("Decrypted message (m_recovered):", m_recovered)
print("Original message: ",m)


f:  [1, 1, -1, 0, -1, 0, 1]
g:  [1, 0, 1, -1, 0, 0, -1]
Inverse of f mod q (Fq): [30  6 38 22 10 36 23]
Inverse of f mod p (Fp): [1 0 0 0 2 2 2]
Print polynomial Multiplication:  [ 9 37 23 29  6 38 22]
Public key (h): [27 29 28  5 18 32 25]
Private key (f): [1, 1, -1, 0, -1, 0, 1]
r:  [1, 1, 0, -1, 0, 0, -1]
Print polynomial Multiplication:  [ 5 37 27 29  3 38 25]
Encrypted message (e): [ 6 37 28 30  3 37 25]
Print polynomial Multiplication:  [ 1  2 11 40 36 39 37]
Print polynomial Multiplication:  [1 0 1 1 0 2 0]
Decrypted message (m_recovered): [ 1  0  1  1  0 -1  0]
Original message:  [ 1  0  1  1  0 -1  0]
